In [ ]:
import os
import mlflow

In [ ]:
experiments = ['ads_local', 'bank_housing', 'biodeg_V25', 'churn_voice_mail_plan', 'credit_A1', 
            'monks1_attr3', 'phishing_having_IP_Address', 'sick_referral_source_other', 'vote_immigration']

fair_experiments = ['adult_sex_Male', 'adult_race_White', 'german_sex_Male', 'compas_sex', 
                    'compas_race_Caucasian', 'ricci_Race_W', 'diabetes_race_Caucasian', 'titanic_sex']

algorithms = ['PL', 'STC', 'CC', 'HLNC', 'OBNC', 'BE', 
              'OBNC-remove-sensitive', 'OBNC-optimize-demographic-parity', 'OBNC-fair']

noise_types = ['flip', 'random', 'bias', 'balanced_bias']

# Saving standard ML datasets results

In [ ]:
for noise_type in noise_types:
    if not os.path.exists(f'noiseinjection_results/{noise_type}'):
        os.makedirs(f'noiseinjection_results/{noise_type}')

for exp in experiments:
    for alg in algorithms:
        name = f'{exp}_{alg}'
        run = mlflow.search_runs(experiment_names=[name], order_by=['start_time DESC'])
        for noise_type in noise_types:
            if alg == 'OBNC-optimize-demographic-parity' or alg == 'OBNC-fair':
                for prob in ['0', '0.5']:
                    df = run.loc[(run['params.noise_type'] == noise_type) & (run['params.prob'] == prob)]
                    df.to_csv(f'noiseinjection_results/{noise_type}/{name}-{prob}.csv', index=False)
            else:
                df = run.loc[run['params.noise_type'] == noise_type]
                df.to_csv(f'noiseinjection_results/{noise_type}/{name}.csv', index=False)


# Saving fairness benchmark datasets

In [ ]:
if not os.path.exists(f'fairness_benchmark_results'):
    os.mkdir(f'fairness_benchmark_results')

for exp in fair_experiments:
    for alg in algorithms:
        name = f'{exp}_{alg}'
        run = mlflow.search_runs(experiment_names=[name], order_by=['start_time DESC'])
        if alg == 'OBNC-optimize-demographic-parity' or alg == 'OBNC-fair':
            for prob in ['0', '0.5']:
                df = run.loc[run['params.prob'] == prob]
                df.to_csv(f'fairness_benchmark_results/{name}-{prob}.csv', index=False)
        else:
            run.to_csv(f'fairness_benchmark_results/{name}.csv', index=False)